### Code flow
1. Load The Dataset
2. Basic Preprocessing
3. Training Process
    - Create the model
    - Forward pass
    - Loss calculation
    - Backprop
    - Params update
4. Model Evaluation (accuracy)

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [ ]:
df.head()

### train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

### scaling - as NN prefers datas to be in same scale

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train

In [ ]:
y_train

### Label Encoding - as NN cant understand alphabets

In [ ]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [ ]:
y_train

### Numpy arrays to PyTorch tensors

In [ ]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [ ]:
X_train_tensor.shape  # 30 features == 30 weights AND 1 neuron == 1 bias in the model

In [ ]:
y_train_tensor.shape

### Important Parameters

In [ ]:
learning_rate = 0.1
epochs = 30

### Defining the model

In [ ]:
class MySimpleNN:
    def __init__(self, X):
        self.weights = torch.rand(X.shape[1], 1, dtype=torch.float64, requires_grad=True)
        self.bias = torch.zeros(1, dtype=torch.float64, requires_grad=True)
        print(f'Weights shape: {self.weights.shape}, Bias shape: {self.bias.shape}')

    def forward(self, X):
        z = torch.matmul(X, self.weights) + self.bias
        y_pred = torch.sigmoid(z)
        return y_pred

    def loss_function(self, y_pred, y_train_tensor):
        # Clamp predictions to avoid log(0)
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

        # Calculate loss
        loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
        return loss


### Training Pipeline

In [ ]:
# model = MySimpleNN(X_train_tensor)
# y_pred = model.forward(X_train_tensor)
# print(y_pred.shape)

In [ ]:
# instantiate model
model = MySimpleNN(X_train_tensor)

# define loop
for epoch in range(1,epochs+1):
    # forward pass
    y_pred = model.forward(X_train_tensor)
    # loss calculate
    loss = model.loss_function(y_pred, y_train_tensor)

    # backward pass
    loss.backward()

    # parameters update
    with torch.no_grad():
        model.weights.data -= learning_rate * model.weights.grad
        model.bias.data    -= learning_rate * model.bias.grad

    # zero gradients - to prevent Gradient Accumulation
    model.weights.grad.zero_()
    model.bias.grad.zero_()

    # print loss in each epoch
    print(f'Epoch: {epoch}, Loss: {loss.item()}')

In [ ]:
model.weights

In [ ]:
model.bias

### Evaluation

In [ ]:
# model evaluation
with torch.no_grad():
    y_pred = model.forward(X_test_tensor)  # value between 0 and 1
    y_pred = (y_pred > 0.5).float() # convert to binary predictions
    accuracy = (y_pred == y_test_tensor).float().mean()
    print(f'Accuracy: {accuracy.item()}')


### Needs few improvement

### How forward automatically works

In [ ]:
from torch import nn

class MyLinear(nn.Linear):
    def forward(self, input):
        print("Inside MyLinear.forward")
        return super().forward(input)

layer = MyLinear(10, 5)
x = torch.randn(2, 10)
out = layer(x) # the class object is called as a function, this triggers the __call__ method
    # prints message → proves __call__ routes to forward

In [ ]:
out